In [ ]:
# Fetch Crypto Data


import requests
import pandas as pd

def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 20,
        'page': 1,
        'sparkline': 'false'
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

crypto_data = fetch_crypto_data()
crypto_data[:3]  # Display first 3 entries for verification


In [ ]:
# Filter and Select Top 10 Cryptos (Excluding Stable Coins)

def filter_stable_coins(data):
    stable_coins = ["tether", "usd-coin", "binance-usd"]
    filtered_data = [coin for coin in data if coin['id'] not in stable_coins]
    return filtered_data

filtered_data = filter_stable_coins(crypto_data)
top_10_cryptos = filtered_data[:10]  # Select top 10

for crypto in top_10_cryptos:
    print(f"{crypto['name']}: Market Cap = ${crypto['market_cap']}")


In [ ]:
# Calculate Monthly Rebalancing

def calculate_weights(cryptos):
    total_market_cap = sum([crypto['market_cap'] for crypto in cryptos])
    for crypto in cryptos:
        crypto['weight'] = crypto['market_cap'] / total_market_cap
    return cryptos

def rebalance_index():
    data = fetch_crypto_data()
    filtered_data = filter_stable_coins(data)
    top_10_cryptos = filtered_data[:10]
    weighted_cryptos = calculate_weights(top_10_cryptos)
    return weighted_cryptos

rebalanced_index = rebalance_index()

for crypto in rebalanced_index:
    print(f"{crypto['name']} ({crypto['symbol']}): Weight = {crypto['weight']*100:.2f}%")


In [ ]:
# Automate Monthly Rebalancing

import schedule
import time

def job():
    rebalanced_index = rebalance_index()
    # Save or update your index data (e.g., save to a CSV file)
    df = pd.DataFrame(rebalanced_index)
    df.to_csv('data/crypto_index.csv', index=False)
    print("Index rebalanced and saved.")

# Schedule the job to run on the first day of every month
schedule.every().month.at("00:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)
